In [0]:
import requests
import pandas as pd
from datetime import datetime

In [0]:
def ExtrairDadosBitcoin():
    url = 'https://api.coinbase.com/v2/prices/spot'
    resposta = requests.get(url)
    return resposta.json()

In [0]:
ExtrairDadosBitcoin()

In [0]:
def ExtrairCotacaoUsdBrl():
    api_key = 'f036381318d146f5a412943af384570f'
    url = f'https://api.currencyfreaks.com/v2.0/rates/latest?apikey={api_key}'
    resposta = requests.get(url)
    return resposta.json()

<h1>Tranformação<h1>

In [0]:
def tratar_dados_bitcoin(dados_json, taxa_usd_brl):
    ValorUsd = float (dados_json['data']['amount'])
    
    CriptoMoeda = dados_json['data']['base']
    MoedaOriginal = dados_json['data']['currency']
    
    ValorBrl = ValorUsd * taxa_usd_brl

    timestamp = datetime.now()

    DadosTratados = [{
      "ValorUsd": ValorUsd,
      "ValorBrl": ValorBrl,
      "CriptoMoeda": CriptoMoeda,
      "MoedaOriginal": MoedaOriginal,
      "taxa_convercao_usd_brl": taxa_usd_brl,
      "timestamp": timestamp
    }]
    return DadosTratados

In [0]:
DadosBitcoin = ExtrairDadosBitcoin()
DadosCotacao = ExtrairCotacaoUsdBrl()

taxa_usd_brl = float(DadosCotacao['rates']['BRL'])

dados_bitcoins_tratados = tratar_dados_bitcoin(DadosBitcoin, taxa_usd_brl)

print(dados_bitcoins_tratados)

<h1>Criando catalogo</h1>

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS pipeline_api_bitcoin
COMMENT 'catalogo teste';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS pipeline_api_bitcoin.lakehouse
COMMENT 'schema teste';

In [0]:
%sql
CREATE VOLUME IF NOT EXISTS pipeline_api_bitcoin.lakehouse.raw_files
comment 'volume teste';

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS pipeline_api_bitcoin.bitcoin_data
COMMENT 'schema para armazenar os dados de bicoins processados';

## crirando DataFrame

In [0]:
df_pandas = pd.DataFrame(dados_bitcoins_tratados)

print (df_pandas)

## salvando em json

In [0]:
#pegando timestamp
event_ts = dados_bitcoins_tratados[0]['timestamp']

#convertendo para string
ts = event_ts.strftime("%Y%M%d_%H%M%S_%f")

#identificando o caminho do arquivo
json_files = f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin.json"

#salvando como json
df_pandas.to_json(json_files, orient='records', date_format='iso', indent=2)
print(f"Dados json gravados em {json_files}")

##Salvando em csv

In [0]:
#identificando o caminho do arquivo
csv_files = f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin.csv"

#salvando como csv
df_pandas.to_csv(csv_files, index=False)
print(f"Dados csv gravados em {csv_files}")

##Salvando em parquet

In [0]:
#identificando o caminho do arquivo
parquet_files = f"/Volumes/pipeline_api_bitcoin/lakehouse/raw_files/bitcoin.parquet"

#salvando como parquet
df_pandas.to_parquet(parquet_files, index=False)
print(f"Dados parquet gravados em {parquet_files}")

##Salvando como delta table

In [0]:
df_spark = spark.createDataFrame(df_pandas)
df_spark.printSchema()

In [0]:
df_spark.write.format("delta").mode("append").saveAsTable("pipeline_api_bitcoin.bitcoin_data.dados_bitcoin")

## consultando Delta table via SQL

In [0]:
%sql
SELECT * FROM pipeline_api_bitcoin.bitcoin_data.dados_bitcoin

##Verificando histórico de Delta Table (Time Travel)

In [0]:
%sql
DESCRIBE HISTORY pipeline_api_bitcoin.bitcoin_data.dados_bitcoin